<a href="https://colab.research.google.com/github/EmilKJohn99/hello/blob/main/amazonreviewsentimentanalysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [2]:

import numpy as np
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [3]:

import pandas as pd
import re
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

In [4]:
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import re
import pyspark

In [5]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('punkt_tab')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [6]:
!kaggle datasets download bittlingmayer/amazonreviews

Dataset URL: https://www.kaggle.com/datasets/bittlingmayer/amazonreviews
License(s): unknown
 94% 463M/493M [00:01<00:00, 369MB/s]
100% 493M/493M [00:01<00:00, 447MB/s]


In [7]:
from zipfile import ZipFile
dataset='/content/amazonreviews.zip'

with ZipFile(dataset,'r') as zip:
  zip.extractall()
  print("dataset has been extracted")

dataset has been extracted


In [8]:
!bunzip2 test.ft.txt.bz2


In [9]:
!bunzip2 train.ft.txt.bz2


In [ ]:
# Read and print the first 10 lines of the file
file_path = "/content/test.ft.txt"

with open(file_path, "r", encoding="utf-8") as file:
    for i in range(10):
        line = file.readline()
        if not line:
            break
        print(line.strip())


__label__2 Great CD: My lovely Pat has one of the GREAT voices of her generation. I have listened to this CD for YEARS and I still LOVE IT. When I'm in a good mood it makes me feel better. A bad mood just evaporates like sugar in the rain. This CD just oozes LIFE. Vocals are jusat STUUNNING and lyrics just kill. One of life's hidden gems. This is a desert isle CD in my book. Why she never made it big is just beyond me. Everytime I play this, no matter black, white, young, old, male, female EVERYBODY says one thing "Who was that singing ?"
__label__2 One of the best game music soundtracks - for a game I didn't really play: Despite the fact that I have only played a small portion of the game, the music I heard (plus the connection to Chrono Trigger which was great as well) led me to purchase the soundtrack, and it remains one of my favorite albums. There is an incredible mix of fun, epic, and emotional songs. Those sad and beautiful tracks I especially like, as there's not too many of th

In [10]:
import csv

input_path = "/content/test.ft.txt"
output_path = "test.csv"

with open(input_path, "r", encoding="utf-8") as infile, open(output_path, "w", newline='', encoding="utf-8") as outfile:
    writer = csv.writer(outfile)
    writer.writerow(["label", "text"])  # CSV headers

    for line in infile:
        line = line.strip()
        if line.startswith("__label__"):
            parts = line.split(" ", 1)
            if len(parts) == 2:
                label = parts[0].replace("__label__", "")
                text = parts[1]
                writer.writerow([label, text])


In [11]:
import csv

input_path = "/content/train.ft.txt"
output_path = "train.csv"

with open(input_path, "r", encoding="utf-8") as infile, open(output_path, "w", newline='', encoding="utf-8") as outfile:
    writer = csv.writer(outfile)
    writer.writerow(["label", "text"])  # CSV headers

    for line in infile:
        line = line.strip()
        if line.startswith("__label__"):
            parts = line.split(" ", 1)
            if len(parts) == 2:
                label = parts[0].replace("__label__", "")
                text = parts[1]
                writer.writerow([label, text])


In [12]:
traindata = pd.read_csv('/content/train.csv', encoding='ISO-8859-1')

In [ ]:
traindata.head()


,label,text
0,2,Stuning even for the non-gamer: This sound tra...
1,2,The best soundtrack ever to anything.: I'm rea...
2,2,Amazing!: This soundtrack is my favorite music...
3,2,Excellent Soundtrack: I truly like this soundt...
4,2,"Remember, Pull Your Jaw Off The Floor After He..."


In [13]:
# Replace label values: 2 -> 1, 1 -> 0
traindata["label"] = traindata["label"].replace({2: 1, 1: 0})

# Preview the updated DataFrame
traindata.head()


,label,text
0,1,Stuning even for the non-gamer: This sound tra...
1,1,The best soundtrack ever to anything.: I'm rea...
2,1,Amazing!: This soundtrack is my favorite music...
3,1,Excellent Soundtrack: I truly like this soundt...
4,1,"Remember, Pull Your Jaw Off The Floor After He..."


In [14]:
testdata=pd.read_csv('/content/test.csv', encoding='ISO-8859-1')

In [15]:
# Replace label values: 2 -> 1, 1 -> 0
testdata["label"] = testdata["label"].replace({2: 1, 1: 0})

# Preview the updated DataFrame
testdata.head()

,label,text
0,1,Great CD: My lovely Pat has one of the GREAT v...
1,1,One of the best game music soundtracks - for a...
2,0,Batteries died within a year ...: I bought thi...
3,1,"works fine, but Maha Energy is better: Check o..."
4,1,Great for the non-audiophile: Reviewed quite a...


In [16]:
def clean_text(text):
    text = re.sub(r'http\S+', '', text)  # remove URLs
    text = re.sub(r'[^a-zA-Z\s]', '', text)  # keep only letters
    text = text.lower()
    return text

In [17]:
traindata['clean_text'] = traindata['text'].apply(clean_text)

In [18]:
testdata['clean_text']=testdata['text'].apply(clean_text)

In [19]:
traindata.rename(columns={"label": "target"}, inplace=True)

In [20]:
testdata.rename(columns={"label": "target"}, inplace=True)

In [21]:
df = pd.concat([traindata, testdata], ignore_index=True)

In [22]:
texts = df['clean_text'].values
labels = df['target'].values

In [23]:
max_words = 10000
max_len = 40

tokenizer = Tokenizer(num_words=max_words, oov_token="<OOV>")
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)
padded_sequences = pad_sequences(sequences, maxlen=max_len, padding='post', truncating='post')


In [24]:
X_train, X_val, y_train, y_val = train_test_split(
    padded_sequences, labels, test_size=0.2, stratify=labels, random_state=42
)


In [26]:
model = Sequential([
    Embedding(input_dim=max_words, output_dim=64, input_length=max_len),
    LSTM(64, return_sequences=False),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

early_stop = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
checkpoint = ModelCheckpoint('best_lstm_model.h5', save_best_only=True, monitor='val_loss')

In [27]:
history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=3,
    batch_size=128,
    callbacks=[early_stop, checkpoint],
    verbose=2
)

Epoch 1/3


25000/25000 - 2359s - 94ms/step - accuracy: 0.9099 - loss: 0.2236 - val_accuracy: 0.9233 - val_loss: 0.1915
Epoch 2/3


25000/25000 - 2321s - 93ms/step - accuracy: 0.9273 - loss: 0.1862 - val_accuracy: 0.9289 - val_loss: 0.1819
Epoch 3/3


25000/25000 - 2361s - 94ms/step - accuracy: 0.9337 - loss: 0.1722 - val_accuracy: 0.9298 - val_loss: 0.1786


In [28]:
loss, accuracy = model.evaluate(X_val, y_val)
print(f"\nFinal Validation Accuracy: {accuracy:.4f}")

25000/25000 ━━━━━━━━━━━━━━━━━━━━ 320s 13ms/step - accuracy: 0.9300 - loss: 0.1782

Final Validation Accuracy: 0.9298
